# 📊 Analyse des données

## 📌 Introduction
Ce notebook a pour objectif d'analyser les données afin d'en extraire des insights pertinents.


### 📂 Importation des données & librairies

In [1]:
from utils import *
import numpy as np
import scipy.stats as stats
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#pour voir les visus, faire tourner les notebooks en local

In [2]:
df = load_cleaned_data()

In [3]:
df.describe()

,Annee,Debit,Hauteur
count,123.000000,123.000000,123.000000
mean,1926.422764,1303.299533,3.896748
std,44.273740,681.342988,1.272337
min,1850.000000,122.435600,1.000000
25%,1886.500000,796.274404,3.100000
50%,1931.000000,1255.005113,3.900000
75%,1961.500000,1616.866106,4.750000
max,1997.000000,3361.255608,7.200000


## 📊 Visualisation des données

### Détermination de la Hauteur Optimale de la Digue

 🎯 Objectif : Nous cherchons à déterminer la hauteur optimale de la digue en analysant la distribution des hauteurs de crues.

 📌 Méthodologie : Calculer le **percentile 95%** des hauteurs de crues.

 💡 Interprétation : Une digue dimensionnée selon le **percentile 95%** couvre 95% des crues historiques. **5% des crues** risquent encore de dépasser cette hauteur, mais cela réduit considérablement les coûts de construction.



In [4]:
data = df["Hauteur"]

distributions = {
    "norm": stats.norm,
    "gumbel_r": stats.gumbel_r,
    "gumbel_l": stats.gumbel_l,
    # "weibull_min": stats.weibull_min,
    "expon": stats.expon,
    "logistic": stats.logistic
}

best_dist = None 
best_stat = float("inf")

#Anderson-Darling adequation sur valeurs extremes
for name, dist in distributions.items():
    params = dist.fit(data)  
    ad_stat, critical_values, significance = stats.anderson(data, dist=name)
    print(f"{name} : Anderson-Darling Stat = {ad_stat:.5f}, seuil critique = {critical_values[2]:.5f}")

    #plus petite stat AD
    if ad_stat < best_stat:
        best_stat = ad_stat
        best_dist = name

print(f"\n✅ Meilleure distribution selon le test d'Anderson-Darling : {best_dist} (AD Stat = {best_stat:.5f})")
plot_fitted_distributions(data, distributions)

norm : Anderson-Darling Stat = 0.29832, seuil critique = 0.76300
gumbel_r : Anderson-Darling Stat = 1.07553, seuil critique = 0.74400
gumbel_l : Anderson-Darling Stat = 2.12803, seuil critique = 0.74400
expon : Anderson-Darling Stat = 25.58989, seuil critique = 1.33400
logistic : Anderson-Darling Stat = 0.36275, seuil critique = 0.65900

✅ Meilleure distribution selon le test d'Anderson-Darling : norm (AD Stat = 0.29832)


In [5]:
dist_finale = distributions[best_dist]
seuil_risque = 0.10  # 10% de crues qui dépassent (arbitraire)
h_optimal = get_optimal_digue_height(df, dist_finale, seuil_risque)

print(f"Hauteur optimale de la digue pour {100*(1-seuil_risque)}% de protection : {h_optimal:.2f} m")

Hauteur optimale de la digue pour 90.0% de protection : 5.52 m


In [6]:
import plotly.express as px

In [7]:
fig = px.histogram(
    df,
    x="Hauteur",
    nbins=20,
    opacity=0.8,
    color_discrete_sequence=["darkblue"],
    title="Distribution de la hauteur des crues"
)

fig.update_layout(
    bargap=0.1,
    xaxis_title="Hauteur de crue (en m)",
    template="plotly_white"
)

fig.show()

In [8]:
fig = px.histogram(
    df,
    x="Debit",
    nbins=20,
    opacity=0.8,
    color_discrete_sequence=["darkred"],
    title="Distribution du débit des crues"
)

fig.update_layout(
    bargap=0.1,
    xaxis_title="Débit de crue (en m³/s)",
    template="plotly_white"
)

fig.show()

In [9]:
fig = px.line(
    df,
    x="Annee",
    y="Hauteur",
    title="Évolution de la hauteur de crue au fil du temps"
)

fig.update_layout(
    xaxis_title="Année",
    yaxis_title="Hauteur de crue (en m)",
    template="plotly_white"
)

fig.update_traces(
    line_color='darkblue'
)

fig.show()

> On observe une certaine périodicité, mise en évidence par la récurrence des pics.

In [10]:
#---seuil optimal---
visualize_distribution_with_threshold(df, h_optimal)

In [11]:
visualize_boxplot(df)

> Aucun outlier particulier n'est détecté. La médiane est d'environ 4 mètres.